In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

2024-04-26 16:16:43.927226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create TableGan Model 

In [2]:
df = pd.read_csv("com_salary.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              909 non-null    object 
 1   OttoneuID         909 non-null    int64  
 2   FG MajorLeagueID  851 non-null    float64
 3   FG MinorLeagueID  902 non-null    object 
 4   MLB Org           896 non-null    object 
 5   Position(s)       909 non-null    object 
 6   Avg Salary        909 non-null    float64
 7   Median Salary     909 non-null    float64
 8   Min Salary        909 non-null    int64  
 9   Max Salary        909 non-null    int64  
 10  Last 10           909 non-null    float64
 11  Roster%           909 non-null    float64
 12  Team              853 non-null    object 
 13  POS               909 non-null    object 
 14  ADP               909 non-null    float64
 15  rPTS              909 non-null    float64
 16  PTS               909 non-null    float64
 1

In [4]:
df.head()

,Name,OttoneuID,FG MajorLeagueID,FG MinorLeagueID,MLB Org,Position(s),Avg Salary,Median Salary,Min Salary,Max Salary,...,POS,ADP,rPTS,PTS,aPOS,Dollars,Adjusted,Cost,PlayerId,value
0,Juan Soto,23717,20123.0,sa906282,NYY,OF,60.31,60.0,34,84,...,OF/DH,10.57,1132.535865,53.429347,21.885623,76.314969,112.280793,NaN,20123,0.000000
1,Mookie Betts,18276,13611.0,sa597889,LAD,2B/SS/OF,58.35,58.0,38,87,...,2B/SS/OF,5.03,1036.172159,40.095221,14.934803,56.030024,82.308998,NaN,13611,0.000000
2,Shohei Ohtani,33600,19755.0,NaN,LAD,Util/SP,56.13,56.0,32,88,...,P/DH,12.73,1038.276653,40.386426,12.178202,53.564628,75.049219,54.0,19755,-0.435372
3,Aaron Judge,18312,15640.0,sa549847,NYY,OF,54.17,54.5,33,70,...,OF/DH,11.16,1066.255227,44.257902,21.885623,67.143525,93.070498,50.0,15640,17.143525
4,Freddie Freeman,5680,5361.0,sa390291,LAD,1B,50.76,51.0,27,74,...,1B,8.81,1062.235872,43.701733,13.809086,58.510819,81.663561,47.0,5361,11.510819


In [5]:
orig_df= df[['ADP', 'Avg Salary', 'Median Salary', 'Min Salary', 'Max Salary', 'rPTS', 'PTS', 'Dollars']]


In [6]:
orig_df.head()

,ADP,Avg Salary,Median Salary,Min Salary,Max Salary,rPTS,PTS,Dollars
0,10.57,60.31,60.0,34,84,1132.535865,53.429347,76.314969
1,5.03,58.35,58.0,38,87,1036.172159,40.095221,56.030024
2,12.73,56.13,56.0,32,88,1038.276653,40.386426,53.564628
3,11.16,54.17,54.5,33,70,1066.255227,44.257902,67.143525
4,8.81,50.76,51.0,27,74,1062.235872,43.701733,58.510819


## Step 1: Define the Generator
The Generator's role in a GAN is to create synthetic data that is indistinguishable from real data. It learns to do this through the adversarial process with the Discriminator. Here, we define a simple neural network model for the


In [7]:
# Define the Generator
class Generator(Model):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.z_dim = z_dim  # Store the dimension as an attribute
        self.dense1 = layers.Dense(64, activation='relu')
        self.out = layers.Dense(z_dim, activation='tanh')

    def call(self, x):
        x = self.dense1(x)
        return self.out(x)



## Step 2: Define the Discriminator
The Discriminator acts as a classifier that tries to distinguish real data from fake data produced by the Generator. This class is also defined with a simple architecture, consisting of one hidden layer.


In [8]:
# Define the Discriminator
class Discriminator(Model):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        return self.out(x)





## Step 3: Define the TableGAN Model
Here we integrate the Generator and Discriminator into a complete GAN model. The TableGAN class manages the training loop where both models are trained in an adversarial setup. This structure includes methods for compiling the model and defining the training step, utilizing TensorFlow's capabilities.


In [11]:
# Define the TableGAN model
class tablegan(Model):
    def __init__(self, input_dim, z_dim):
        super(T=tablegan, self).__init__()
        self.generator = Generator(z_dim)
        self.discriminator = Discriminator(input_dim)

    def compile(self, g_optimizer, d_optimizer, loss_function):
        super(TableGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_function = loss_function

    def train_step(self, real_data):
        batch_size = tf.shape(real_data)[0]
        # Use the stored z_dim directly from the generator
        z_dim = self.generator.z_dim

        random_latent_vectors = tf.random.normal(shape=(batch_size, z_dim))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_data = self.generator(random_latent_vectors)
            real_output = self.discriminator(real_data)
            fake_output = self.discriminator(generated_data)

            gen_loss = self.loss_function(tf.ones_like(fake_output), fake_output)
            real_loss = self.loss_function(tf.ones_like(real_output), real_output)
            fake_loss = self.loss_function(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {'gen_loss': gen_loss, 'disc_loss': disc_loss}

# Example of setting up optimizers and loss functions
g_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Create a TableGAN instance and compile
table_gan = tablegan(input_dim=20, z_dim=100)
table_gan.compile(g_optimizer=g_optimizer, d_optimizer=d_optimizer, loss_function=loss_function)

SyntaxError: positional argument follows keyword argument (3809538683.py, line 4)

## Step 4: Data Loading and Preprocessing
Before training the model, it's necessary to load and preprocess your data. This step involves reading a CSV file, detecting numerical and categorical columns, and applying appropriate transformations such as scaling for numerical data and one-hot encoding for categorical data.


In [10]:
# Load and preprocess the data
data = orig_df  # Ensure orig_df is your DataFrame loaded previously

# Selecting numerical and categorical columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

# Define preprocessing for numerical columns: impute missing values then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical columns: impute missing values then encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Create a ColumnTransformer to apply the above transformations appropriately
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_cols),
        ('cat', cat_pipeline, categorical_cols)
    ])

# Apply transformations to your data
processed_data = preprocessor.fit_transform(data)






## Step 5: Initialize and Train the TableGAN Model
Finally, initialize the TableGAN model with dimensions derived from the preprocessed data, compile it with chosen optimizers and loss function, and train the model. Here, we specify the learning rates, the loss function, and the training parameters such as the number of epochs and batch size.


In [11]:
# Assuming z_dim is defined earlier as used in initializing TableGAN
z_dim = 50  # Update this if it's different

# Start timing the generation process
start_time = time.time()

# Generate synthetic data
random_latent_vectors = tf.random.normal(shape=(1000, z_dim))  # Generate 1000 samples
synthetic_data = tablegan.generator(random_latent_vectors).numpy()

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Synthetic data generation completed in {elapsed_time:.2f} seconds.")

# Convert the synthetic data to a pandas DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=[f"feature_{i+1}" for i in range(synthetic_data.shape[1])])

# Print the DataFrame shape and head to verify
print("Shape of synthetic DataFrame:", synthetic_df.shape)
print("First few rows of the synthetic DataFrame:")
print(synthetic_df.head())

NameError: name 'tablegan' is not defined

## Step 5: Save to CSV
Save the synthetic data to a CSV file. This file can be used for further analysis, sharing, or as input to other analytical tools and systems.


In [ ]:
# Write to CSV
synthetic_df.to_csv('synthetic_data.csv', index=False)

print("Synthetic data written to synthetic_data.csv successfully!")


Step 5 Load Datasets: Original and Synthetic

In [1]:
#Orig_data = orig_df
#synthetic_df

In [2]:
##Define Evaluation Functions
# Evaluation metrics implementation

# Fidelity Metrics
def evaluate_fidelity(real_data, synthetic_data, continuous_columns, categorical_columns):
    ks_results = {col: ks_2samp(real_data[col], synthetic_data[col]) for col in continuous_columns}
    chi_squared_results = {col: chi2_contingency(pd.crosstab(real_data[col], synthetic_data[col]))[:2] for col in categorical_columns}
    return {'KS Test': ks_results, 'Chi-Squared Test': chi_squared_results}

# Utility Metrics
def evaluate_predictive_performance(real_data, synthetic_data, target_column, test_size=0.3, random_state=42):
    X_real = real_data.drop(columns=[target_column])
    y_real = real_data[target_column]
    X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(X_real, y_real, test_size=test_size, random_state=random_state)
    X_synthetic = synthetic_data.drop(columns=[target_column])
    y_synthetic = synthetic_data[target_column]
    model = RandomForestClassifier(n_estimators=100, random_state=random_state)
    model.fit(X_synthetic, y_synthetic)
    predictions = model.predict(X_test_real)
    return {
        'Accuracy': accuracy_score(y_test_real, predictions),
        'ROC AUC': roc_auc_score(y_test_real, model.predict_proba(X_test_real)[:, 1]),
        'F1 Score': f1_score(y_test_real, predictions)
    }

def informativeness_test(real_data, synthetic_data, test_size=0.3, random_state=42):
    real_data['is_real'] = 1
    synthetic_data['is_real'] = 0
    combined_data = pd.concat([real_data, synthetic_data], ignore_index=True)
    X = combined_data.drop(columns=['is_real'])
    y = combined_data['is_real']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    classifier = RandomForestClassifier(n_estimators=100, random_state=random_state)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    return {
        'Accuracy': accuracy_score(y_test, predictions),
        'ROC AUC': roc_auc_score(y_test, classifier.predict_proba(X_test)[:, 1]),
        'F1 Score': f1_score(y_test, predictions)
    }

In [3]:
#Evaluate Fidelity
# Identify columns to apply metrics
continuous_columns = real_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = real_data.select_dtypes(include=['object']).columns.tolist()

# Evaluate fidelity
fidelity_results = evaluate_fidelity(real_data, synthetic_data, continuous_columns, categorical_columns)
print("Fidelity Results:", fidelity_results)


NameError: name 'real_data' is not defined

In [ ]:
# Evaluate predictive performance
predictive_performance_results = evaluate_predictive_performance(real_data, synthetic_data, 'target_column')
print("Predictive Performance Results:", predictive_performance_results)


In [ ]:
# Evaluate informativeness
informativeness_results = informativeness_test(real_data, synthetic_data)
print("Informativeness Results:", informativeness_results)
